In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

In [2]:
df_train = pd.read_csv('../dataset/processed/train/metrics/metrics.csv')
df_test = pd.read_csv('../dataset/processed/test/metrics/metrics.csv')
df_test_label = pd.read_csv('../dataset/processed/test/label.csv')
df_test_label = df_test_label[df_test_label['label']==1]
root_cause_label = pd.read_csv('../dataset/processed/test/label.csv')

In [3]:
scaler = MinMaxScaler()
df_train.iloc[:, 1:] = scaler.fit_transform(df_train.iloc[:, 1:].values)
df_test.iloc[:, 1:] = scaler.transform(df_test.iloc[:, 1:].values)

In [4]:
device = ['cpu', 'mem']
node = ['carts', 'carts_db', 'catalogue', 'catalogue_db', 'front_end', 
        'ip5', 'ip6', 'ip7', 'orders', 'orders_db', 
        'payment', 'queue_master', 'rabbitmq', 'session_db', 'shipping', 
        'user', 'user_db']

In [5]:
df_test_label = df_test_label[df_test_label['label']==1]
df_test_label[['timestamp', 'root_cause']].to_csv('../dataset/processed/root_cause/injection_info.csv')
df_test_label.head()

,timestamp,label,root_cause
105,2022-03-26 10:15:00,1,cpu_usage_carts
106,2022-03-26 10:16:00,1,cpu_usage_carts
107,2022-03-26 10:17:00,1,cpu_usage_carts
108,2022-03-26 10:18:00,1,cpu_usage_carts
109,2022-03-26 10:19:00,1,cpu_usage_carts


In [6]:
for i in range(len(df_test_label)):
    timestamp = df_test_label['timestamp'].values[i]
    root_cause = df_test_label['root_cause'].values[i]
    
    info = pd.DataFrame(columns=['device', 'node', 'predict', 'real'])
    for d in device:
        for n in node:
            predict = df_train[d+'_usage_'+n].mean()
            real = df_test[df_test['timestamp']==timestamp][d+'_usage_'+n].values[0]
            info.loc[len(info)] = [d, n, predict, real]
    info.to_csv('../dataset/processed/root_cause/'+timestamp+'.csv', index=False)